# CLASSICAL OPTICS

- Source Analysis
- Metadata Extraction
- Hex Dump
- Histogram
- FFT (Power Spectra & Phase)
- Spatial Frequency
- Error Level Analysis
- ELA Loss Curve
- Compression Ghost
- RGB2HSV
- Lumiance Gradient
- Echo Edge Detection
- Noise / Grain Seperation
- Frequency Split
- PCA
- EXR Transport

In [ ]:
import sys, warnings
sys.path.append('..')
warnings.filterwarnings('ignore')

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from arch import utils, metadata, hex, hist, transport, ela, filter, frequency, pca

In [ ]:
PATH = Path('data/face.jpg')
init_image = utils.load_image(PATH)

---

In [ ]:
md = {}
md.update(metadata.file_metadata(PATH))
md.update(metadata.exif_data(PATH))

[f'{l.upper()}: {md[l]}' for l in md]

In [ ]:
with open(PATH, 'rb') as image_file:
    data = image_file.read()
hex.hexdump(data, 512)

---

In [ ]:
pixel_count, stat = hist.process(init_image)

print(f"""
Image Spec: {np.array(init_image).shape}
Pixel count: {stat[0]}
Min: {stat[1]} | Max: {stat[2]}
Least Frequent: {stat[3]}
Most Frequent: {stat[4]}
Mean: {stat[5]}
Deviation: {stat[6]}
Median: {stat[7]}
Nonzero: {stat[8]}
Empty Bins: {stat[9]}
Unique Colours: {stat[10]}
""")

In [ ]:
n_img = utils.image2array(init_image)
mag, phase, fft = frequency.calc_fft(n_img)

In [ ]:
nx, ny = mag.shape
x = np.arange(-mag.shape[0]//2, mag.shape[0]//2)
y = np.arange(-mag.shape[1]//2, mag.shape[1]//2)
X, Y = np.meshgrid(x, y)
dist = np.sqrt(X**2 + Y**2)

hist, bins = np.histogram(dist.flatten(), bins=np.arange(dist.max()), weights=mag.flatten())

count = hist[1:].argmax()
print('Freqency at which power is greatest:', count)

In [ ]:
width = 0.25
alpha = 0.5

_, ax = plt.subplots(1, 5, figsize=(25, 5))
ax[0].plot(pixel_count[0], pixel_count[1], lw=0.5)
ax[0].bar(pixel_count[0], pixel_count[1], color='k', width=width)
ax[0].vlines(pixel_count[1].argmax(), 0, pixel_count[1].max(), linewidth=width * 5, color='r', label='Max Luminance')

ax[0].set_title('Luminance Distribution')
ax[0].set_xlabel('Pixel Value'), ax[0].set_ylabel('Count')
ax[0].set_xlim([-1, 255]), ax[0].legend()
ax[0].grid(alpha=0.25)

ax[1].bar(pixel_count[0], pixel_count[2], color='r', width=width, label='Red Channel')
ax[1].bar(pixel_count[0], pixel_count[3], color='g', width=width, label='Green Channel')
ax[1].bar(pixel_count[0], pixel_count[4], color='b', width=width, label='Blue Channel')
ax[1].plot(pixel_count[0], pixel_count[2], 'r', lw=0.5)
ax[1].plot(pixel_count[0], pixel_count[3], 'g', lw=0.5)
ax[1].plot(pixel_count[0], pixel_count[4], 'b', lw=0.5)
ax[1].set_title('Channel Distribution')
ax[1].set_xlabel('Pixel Value'), ax[1].set_ylabel('Count')
ax[1].set_xlim([-1, 255]), ax[1].legend()
ax[1].grid(alpha=0.25)

ax[2].imshow(np.log(mag), 'binary')
ax[2].set_title('Power Spectra')

ax[3].imshow(phase, 'binary')
ax[3].set_title('Phase')

ax[4].plot(bins[:-1], hist, lw=1)
ax[4].vlines(count+1, 0, hist[1:].max(), linewidth=1, color='r')
ax[4].set_title('Spatial Frequency')
ax[4].set_xlabel('Distance from Origin'), ax[4].set_ylabel('Power')
ax[4].set_xlim([1,int(dist.max())]), ax[4].set_ylim([0,int(hist[1:].max())])
ax[4].grid(alpha=0.25)

plt.tight_layout()

---

In [ ]:
loss, qm = ela.loss_curve(init_image)
ghost = ela.compute(init_image, qm)

In [ ]:
plt.figure(figsize=(15,5))
ela.plot_curve(loss, qm)
# utils.array2image(ghost)

---

In [ ]:
hsv = filter.rgb2hsv(n_img)
lum = filter.luminance_gradient(n_img, intensity=0.35)
echo = filter.echo_edge(n_img, radius=2, contrast=0.85)
noise = filter.noise_seperation(n_img, mode='median', radius=2, sigma=3)

In [ ]:
mask = frequency.mask_fft(fft, radius=2, blur=25)
low = frequency.decode_low(fft, mask)
high = frequency.decode_high(fft, mask)

In [ ]:
n_comp = pca.extract_components(n_img)
# utils.contact_layer(n_comp, 3, 3).resize((1000,1000))

---

### WRITE

In [ ]:
aov = {'error_level': ghost,
       'hsv': hsv,
       'luminance_gradient': lum,
       'echo_edge': echo,
       'noise_seperation': noise,
       'low_freq': low,
       'high_freq': high,
       }

header = transport.write(n_img, aov, 'test.exr')
header

---

### READ

In [ ]:
layers = transport.read('test.exr')

In [ ]:
label = [l[0] for l in layers]
data = [l[1] for l in layers]

utils.contact_layer(data, 2, 4, label)